# Heating Mesh Tally on CAD geometry made from Shapes

This constructs a reactor geometry from 3 Shape objects each made from points.

The Shapes made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This makes a 3D geometry and material for the centre column

In [ ]:
import paramak

center_column = paramak.RotateMixedShape(
    points=[
        (50, 600, 'straight'),
        (150, 600, 'spline'),
        (100, 0, 'spline'),
        (150, -600, 'straight'),
        (50, -600, 'straight')
    ],
    name='center_column'
)

This makes a 3D geometry and material for breeder blanket. The azimuth_placement_angle argument is used to repeat the geometry around the Z axis at specified angles.

In [ ]:
blanket = paramak.RotateSplineShape(
    points=[
        (600, 0),
        (600, -20),
        (500, -300),
        (400, -300),
        (400, 0),
        (400, 300),
        (500, 300),
        (600, 20)
    ],
    name='blanket',
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315],
)

This makes a reactor object from the three components

In [ ]:
my_reactor = paramak.Reactor([blanket, center_column])

my_reactor.show()

This section forms the neutronics model by combining the 3D model, the plasma source and some assigned materials. Additionally, the tallies to record the heating are specified.

In [ ]:
my_reactor.export_dagmc_h5m()

This next section combines the geometry with the materials and specifies a few mesh tallies

In [ ]:
# makes use of the previously created neutronics geometry (h5m file) and assigns
# actual materials to the material tags. 

import openmc_dagmc_wrapper as odw

import neutronics_material_maker as nmm

# this links the material tags in the dagmc h5m file with materials.
# these materials are input as strings so they will be looked up in the
# neutronics material maker package
material_tag_to_material_dict = {
    'mat_blanket': nmm.Material.from_library(name='DT_plasma'),
    'mat_center_column': nmm.Material.from_library(name='Li4SiO4'),
}

geometry = odw.Geometry(h5m_filename='dagmc.h5m')

materials = odw.Materials(
    h5m_filename='dagmc.h5m',
    correspondence_dict=material_tag_to_material_dict
)

This section sets the the neutronics results to record (know as tallies).

In [ ]:
from dagmc_bounding_box import DagmcBoundingBox

# finds bounding box size from the geometry size
corners = DagmcBoundingBox('dagmc.h5m').corners()

tally1 = odw.MeshTally3D(
    mesh_resolution=(100, 100, 100),
    bounding_box= corners,
    tally_type="(n,Xa)",
)


import openmc
tallies = openmc.Tallies([tally1])

Sets simulation intensity, combines the geometry, materials, tallies and settings into a single object and runs the simulation

In [ ]:
import openmc_plasma_source as ops

settings = odw.FusionSettings()
settings.batches = 4
settings.particles = 1000

# assigns a ring source of DT energy neutrons to the source using the
# openmc_plasma_source package, more details here
# https://github.com/fusion-energy/openmc-plasma-source/
settings.source = ops.FusionRingSource(fuel="DT", radius=350)


my_model = openmc.Model(
    materials=materials,
    geometry=geometry,
    settings=settings,
    tallies=tallies
)
!rm *.h5  # just removes old summary.h5 or statepoint files
statepoint_file = my_model.run()

The following section opens the statepoint and converts the tally result into a VTK file. VTK files can be opened in paraview

In [ ]:
from openmc_mesh_tally_to_vtk import write_mesh_tally_to_vtk
# importing a package for converting regular mesh tallies to vtk files
# more details here https://github.com/fusion-energy/openmc_mesh_tally_to_vtk


# assumes you have a statepoint file from the OpenMC simulation
statepoint = openmc.StatePoint('statepoint.4.h5')

# this shows the tallies present in the statepoint file
print(statepoint.tallies)

# loads up a tally from the statepoint using it's name
my_tally = statepoint.get_tally(name='(n,Xa)_on_3D_mesh')

# converts the tally result into a VTK file
write_mesh_tally_to_vtk(
    tally=my_tally,
    filename = "vtk_file_from_openmc_mesh.vtk",
)

The next section produces download links for:

- vtk files that contain the 3D mesh results (open with Paraview)
- png images that show the resuls of the 2D mesh tally

In [ ]:
from IPython.display import FileLink
display(FileLink('vtk_file_from_openmc_mesh.vtk'))